In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '390 fBodyAccJerk-bandsEnergy()-1,16',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 40),
            classifier_block(40, 30),
            classifier_block(30, 25),
            classifier_block(25, 25),
            nn.Linear(25, 24)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_13 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_14 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_15 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_16 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_17 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_18 = GAN_data[(GAN_data['Subject'] == 11) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_19 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_20 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_21 = GAN_data[(GAN_data['Subject'] == 14) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_22 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_23 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_24 = GAN_data[(GAN_data['Subject'] == 17) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18, X_19, X_20, X_21, X_22, X_23, X_24))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12) + [12] * len(X_13) + [13] * len(X_14) + [14] * len(X_15) + [15] * len(X_16) + [16] * len(X_17) + [17] * len(X_18) + [18] * len(X_19) + [19] * len(X_20) + [20] * len(X_21) + [21] * len(X_22) + [22] * len(X_23) + [23] * len(X_24)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14, 17]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 15.937380075454712, Final Batch Loss: 3.235614776611328
Epoch 2, Loss: 15.838646650314331, Final Batch Loss: 3.1604206562042236
Epoch 3, Loss: 15.839359521865845, Final Batch Loss: 3.1766059398651123
Epoch 4, Loss: 15.766544103622437, Final Batch Loss: 3.1313016414642334
Epoch 5, Loss: 15.759380578994751, Final Batch Loss: 3.1677663326263428
Epoch 6, Loss: 15.655831336975098, Final Batch Loss: 3.109492778778076
Epoch 7, Loss: 15.57589340209961, Final Batch Loss: 3.1090586185455322
Epoch 8, Loss: 15.50944471359253, Final Batch Loss: 3.147876262664795
Epoch 9, Loss: 15.2925283908844, Final Batch Loss: 3.0778558254241943
Epoch 10, Loss: 15.008805274963379, Final Batch Loss: 2.959566831588745
Epoch 11, Loss: 14.978736877441406, Final Batch Loss: 3.174306869506836
Epoch 12, Loss: 14.5722975730896, Final Batch Loss: 2.9933362007141113
Epoch 13, Loss: 14.149859428405762, Final Batch Loss: 2.804372787475586
Epoch 14, Loss: 13.734985828399658, Final Batch Loss: 2.6676602363586426

Epoch 119, Loss: 7.556719541549683, Final Batch Loss: 1.3809492588043213
Epoch 120, Loss: 7.407825708389282, Final Batch Loss: 1.3216485977172852
Epoch 121, Loss: 7.701578497886658, Final Batch Loss: 1.5736013650894165
Epoch 122, Loss: 7.665084362030029, Final Batch Loss: 1.6203268766403198
Epoch 123, Loss: 7.688673377037048, Final Batch Loss: 1.5337859392166138
Epoch 124, Loss: 7.564011335372925, Final Batch Loss: 1.4941507577896118
Epoch 125, Loss: 7.771791100502014, Final Batch Loss: 1.5619646310806274
Epoch 126, Loss: 7.4590864181518555, Final Batch Loss: 1.4571233987808228
Epoch 127, Loss: 7.671635866165161, Final Batch Loss: 1.6422141790390015
Epoch 128, Loss: 7.920945405960083, Final Batch Loss: 1.9721649885177612
Epoch 129, Loss: 7.309614896774292, Final Batch Loss: 1.370534062385559
Epoch 130, Loss: 7.8140387535095215, Final Batch Loss: 1.799122929573059
Epoch 131, Loss: 7.541677474975586, Final Batch Loss: 1.6175528764724731
Epoch 132, Loss: 7.63666558265686, Final Batch Loss

Epoch 237, Loss: 6.265029072761536, Final Batch Loss: 1.2797572612762451
Epoch 238, Loss: 6.449880123138428, Final Batch Loss: 1.5597515106201172
Epoch 239, Loss: 6.1014145612716675, Final Batch Loss: 1.2609460353851318
Epoch 240, Loss: 5.682827174663544, Final Batch Loss: 0.8262578845024109
Epoch 241, Loss: 5.871882617473602, Final Batch Loss: 0.9991458058357239
Epoch 242, Loss: 6.264104843139648, Final Batch Loss: 1.395630955696106
Epoch 243, Loss: 5.889220356941223, Final Batch Loss: 1.0054367780685425
Epoch 244, Loss: 6.039827227592468, Final Batch Loss: 1.2137516736984253
Epoch 245, Loss: 5.9927942752838135, Final Batch Loss: 1.2187166213989258
Epoch 246, Loss: 6.385956048965454, Final Batch Loss: 1.4761635065078735
Epoch 247, Loss: 6.361606597900391, Final Batch Loss: 1.5796210765838623
Epoch 248, Loss: 5.888304591178894, Final Batch Loss: 1.0273126363754272
Epoch 249, Loss: 6.287814617156982, Final Batch Loss: 1.500476598739624
Epoch 250, Loss: 6.40536105632782, Final Batch Loss

Epoch 352, Loss: 5.0750232338905334, Final Batch Loss: 0.8376269936561584
Epoch 353, Loss: 5.36143171787262, Final Batch Loss: 1.2447483539581299
Epoch 354, Loss: 5.382011711597443, Final Batch Loss: 1.2122623920440674
Epoch 355, Loss: 5.125693380832672, Final Batch Loss: 1.0697342157363892
Epoch 356, Loss: 4.886724591255188, Final Batch Loss: 0.7198971509933472
Epoch 357, Loss: 5.417776048183441, Final Batch Loss: 1.1685751676559448
Epoch 358, Loss: 5.22874128818512, Final Batch Loss: 1.0814683437347412
Epoch 359, Loss: 5.234933018684387, Final Batch Loss: 1.074804425239563
Epoch 360, Loss: 5.542828559875488, Final Batch Loss: 1.373955488204956
Epoch 361, Loss: 5.212827444076538, Final Batch Loss: 1.1800851821899414
Epoch 362, Loss: 5.180410325527191, Final Batch Loss: 0.9596810936927795
Epoch 363, Loss: 5.2441911697387695, Final Batch Loss: 1.0934747457504272
Epoch 364, Loss: 5.070677876472473, Final Batch Loss: 1.0006989240646362
Epoch 365, Loss: 5.154768228530884, Final Batch Loss:

Epoch 470, Loss: 4.680335760116577, Final Batch Loss: 1.0522072315216064
Epoch 471, Loss: 4.3410614132881165, Final Batch Loss: 0.6707324981689453
Epoch 472, Loss: 4.678810894489288, Final Batch Loss: 1.031516432762146
Epoch 473, Loss: 4.9865840673446655, Final Batch Loss: 1.3513213396072388
Epoch 474, Loss: 4.5654279589653015, Final Batch Loss: 0.7094364166259766
Epoch 475, Loss: 4.711355745792389, Final Batch Loss: 1.0690332651138306
Epoch 476, Loss: 4.373522758483887, Final Batch Loss: 0.7072591185569763
Epoch 477, Loss: 5.0230395793914795, Final Batch Loss: 1.3021043539047241
Epoch 478, Loss: 4.403699815273285, Final Batch Loss: 0.7119150757789612
Epoch 479, Loss: 4.779449045658112, Final Batch Loss: 1.1492596864700317
Epoch 480, Loss: 4.615052402019501, Final Batch Loss: 0.9159550666809082
Epoch 481, Loss: 4.82598477602005, Final Batch Loss: 1.074839472770691
Epoch 482, Loss: 5.056930422782898, Final Batch Loss: 1.375945806503296
Epoch 483, Loss: 4.767443418502808, Final Batch Los

...

Epoch 4537, Loss: 1.7959008812904358, Final Batch Loss: 0.6057360768318176
Epoch 4538, Loss: 1.9117180407047272, Final Batch Loss: 0.6859598755836487
Epoch 4539, Loss: 1.2167630270123482, Final Batch Loss: 0.06574303656816483
Epoch 4540, Loss: 1.4526012241840363, Final Batch Loss: 0.20804288983345032
Epoch 4541, Loss: 1.344199150800705, Final Batch Loss: 0.11685365438461304
Epoch 4542, Loss: 1.3385108970105648, Final Batch Loss: 0.051686253398656845
Epoch 4543, Loss: 1.6538376212120056, Final Batch Loss: 0.39232465624809265
Epoch 4544, Loss: 1.7836168706417084, Final Batch Loss: 0.4495803415775299
Epoch 4545, Loss: 1.3354259729385376, Final Batch Loss: 0.13686522841453552
Epoch 4546, Loss: 1.4401096850633621, Final Batch Loss: 0.23881883919239044
Epoch 4547, Loss: 1.5220265686511993, Final Batch Loss: 0.24050727486610413
Epoch 4548, Loss: 1.7252914309501648, Final Batch Loss: 0.5042794346809387
Epoch 4549, Loss: 1.797893911600113, Final Batch Loss: 0.5371542572975159
Epoch 4550, Loss: 

Epoch 4652, Loss: 1.4926857650279999, Final Batch Loss: 0.31707528233528137
Epoch 4653, Loss: 1.8978899419307709, Final Batch Loss: 0.6415185928344727
Epoch 4654, Loss: 1.6471637785434723, Final Batch Loss: 0.3913722038269043
Epoch 4655, Loss: 1.5062413439154625, Final Batch Loss: 0.09036274999380112
Epoch 4656, Loss: 1.3932485282421112, Final Batch Loss: 0.049750059843063354
Epoch 4657, Loss: 1.6235210001468658, Final Batch Loss: 0.39148879051208496
Epoch 4658, Loss: 1.4918900281190872, Final Batch Loss: 0.24011816084384918
Epoch 4659, Loss: 1.70378777384758, Final Batch Loss: 0.42004913091659546
Epoch 4660, Loss: 1.8739351034164429, Final Batch Loss: 0.4563877284526825
Epoch 4661, Loss: 1.4624264985322952, Final Batch Loss: 0.09192900359630585
Epoch 4662, Loss: 1.7628569304943085, Final Batch Loss: 0.38380399346351624
Epoch 4663, Loss: 1.360618695616722, Final Batch Loss: 0.09064634144306183
Epoch 4664, Loss: 1.5213534533977509, Final Batch Loss: 0.17719149589538574
Epoch 4665, Loss:

Epoch 4764, Loss: 1.3181604892015457, Final Batch Loss: 0.08824069797992706
Epoch 4765, Loss: 1.6328075230121613, Final Batch Loss: 0.3384116590023041
Epoch 4766, Loss: 1.4635802954435349, Final Batch Loss: 0.1685890406370163
Epoch 4767, Loss: 1.6410718858242035, Final Batch Loss: 0.376504123210907
Epoch 4768, Loss: 1.6540104150772095, Final Batch Loss: 0.3832504451274872
Epoch 4769, Loss: 1.3623510748147964, Final Batch Loss: 0.18857689201831818
Epoch 4770, Loss: 1.3561007231473923, Final Batch Loss: 0.11581166088581085
Epoch 4771, Loss: 1.560865968465805, Final Batch Loss: 0.31873688101768494
Epoch 4772, Loss: 1.549048274755478, Final Batch Loss: 0.2719821333885193
Epoch 4773, Loss: 1.8771938979625702, Final Batch Loss: 0.5844098329544067
Epoch 4774, Loss: 1.6636315286159515, Final Batch Loss: 0.3811592161655426
Epoch 4775, Loss: 1.757193773984909, Final Batch Loss: 0.5177679657936096
Epoch 4776, Loss: 1.5711308717727661, Final Batch Loss: 0.35538095235824585
Epoch 4777, Loss: 1.6290

Epoch 4875, Loss: 1.4723033010959625, Final Batch Loss: 0.13981953263282776
Epoch 4876, Loss: 1.5574347078800201, Final Batch Loss: 0.27486488223075867
Epoch 4877, Loss: 1.7846229374408722, Final Batch Loss: 0.48560264706611633
Epoch 4878, Loss: 1.5859536230564117, Final Batch Loss: 0.3776788115501404
Epoch 4879, Loss: 1.5126896500587463, Final Batch Loss: 0.28425973653793335
Epoch 4880, Loss: 1.4493619501590729, Final Batch Loss: 0.20003843307495117
Epoch 4881, Loss: 1.5925387144088745, Final Batch Loss: 0.27369818091392517
Epoch 4882, Loss: 1.329044371843338, Final Batch Loss: 0.17108884453773499
Epoch 4883, Loss: 1.5584344863891602, Final Batch Loss: 0.3970637917518616
Epoch 4884, Loss: 1.4926426708698273, Final Batch Loss: 0.3132566809654236
Epoch 4885, Loss: 1.2397691532969475, Final Batch Loss: 0.07684733718633652
Epoch 4886, Loss: 1.526261329650879, Final Batch Loss: 0.3180946409702301
Epoch 4887, Loss: 1.6051425337791443, Final Batch Loss: 0.29118695855140686
Epoch 4888, Loss: 

Epoch 4991, Loss: 1.6779500246047974, Final Batch Loss: 0.40050381422042847
Epoch 4992, Loss: 1.5949321389198303, Final Batch Loss: 0.3211595118045807
Epoch 4993, Loss: 1.3985361903905869, Final Batch Loss: 0.17456196248531342
Epoch 4994, Loss: 1.6181000769138336, Final Batch Loss: 0.3325694501399994
Epoch 4995, Loss: 1.4676940329372883, Final Batch Loss: 0.058357227593660355
Epoch 4996, Loss: 1.3499031066894531, Final Batch Loss: 0.08158093690872192
Epoch 4997, Loss: 1.4322989135980606, Final Batch Loss: 0.07290883362293243
Epoch 4998, Loss: 1.5558934211730957, Final Batch Loss: 0.34720054268836975
Epoch 4999, Loss: 1.3225022703409195, Final Batch Loss: 0.15284039080142975
Epoch 5000, Loss: 1.3336227014660835, Final Batch Loss: 0.0535629466176033


In [9]:
softmax = nn.Softmax(dim = 1)
model.train()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  1  0  0  0  0  0  2  0  0  0  0  0  1  0  0  1]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 11  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 12  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1 12  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 10  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  2  0  0  0  0  0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0

# Train on Fake Test on Real

In [10]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [11]:
gen = Generator(z_dim = 111)
load_model(gen, "3 Label 8 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=111, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=32, bias=True)
    (4): Tanh()
  )
)

In [30]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 8)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 3:
        fake_labels.append(9)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 3:
        fake_labels.append(10)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 3:
        fake_labels.append(11)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 4:
        fake_labels.append(12)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 4:
        fake_labels.append(13)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 4:
        fake_labels.append(14)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 5:
        fake_labels.append(15)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 5:
        fake_labels.append(16)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 5:
        fake_labels.append(17)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 6:
        fake_labels.append(18)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 6:
        fake_labels.append(19)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 6:
        fake_labels.append(20)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 7:
        fake_labels.append(21)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 7:
        fake_labels.append(22)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 7:
        fake_labels.append(23)
        
fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [31]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5, zero_division = 0))

[[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  4  0  0  0  0  0  2  0  0  0  0  0  1  0  0  1]
 [ 0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  6  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1 10  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0